In [1]:
import requests
import pprint
import json
import pandas as pd
from datetime import datetime,  timedelta
import tqdm 
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

key = '7bBeVcu1HiKEYVutnwGstfISHvCJKzXcizQ2iFZlhBCaQobHPqOod8G%2BcwO05R81TptPWRtVcKUSqmhqOqCGXQ%3D%3D'


# 1. 국토교통부_(TAGO)_버스정류소정보

## 1-1. 도시코드 목록 조회

In [2]:
def get_city_code(key):
    
    url = 'https://apis.data.go.kr/1613000/BusSttnInfoInqireService/getCtyCodeList?serviceKey=%s&_type=json'%key    
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    data = json_ob['response']['body']['items']['item']
    city_code_df = pd.json_normalize(data)
    return city_code_df

In [3]:
city_code_df = get_city_code(key)

## 1-2. 정류소번호 목록조회

In [4]:
def get_api_conts_2(key,city_code):
    
    url = 'https://apis.data.go.kr/1613000/BusSttnInfoInqireService/getSttnNoList?serviceKey=%s&pageNo=1&numOfRows=1000000&_type=json&cityCode=%s'%(key,str(city_code) )
    
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    data = json_ob['response']['body']['items']['item']
    return data

In [5]:
stop_num = []
for i in tqdm(city_code_df['citycode']):
    try:
        js = get_api_conts_2(key,i)

        for item in js:
            item['citycode'] = i
            stop_num.append(item)
    except:
        print(i)
        pass


 36%|████████████████████████████▊                                                    | 47/132 [00:33<00:47,  1.80it/s]

32370


100%|████████████████████████████████████████████████████████████████████████████████| 132/132 [01:35<00:00,  1.39it/s]


In [6]:
stop_num_df = pd.json_normalize(stop_num)

In [7]:
stop_num_df

,gpslati,gpslong,nodeid,nodenm,nodeno,citycode
0,36.63289,127.49071,SJB270000013,용두사지철당간,1611,12
1,36.629718,127.491301,SJB270000016,육거리,1614,12
2,36.627695,127.491016,SJB270000017,육거리시장,1625,12
3,36.627554,127.490673,SJB270000018,육거리시장,1626,12
4,36.629626,127.491113,SJB270000019,육거리.산림조합,1613,12
...,...,...,...,...,...,...
174418,35.589753,128.007505,TSB398000166,양지저전,3980166,38400
174419,35.604705,128.020357,TSB398000167,고삼,3980167,38400
174420,35.604661,128.020068,TSB398000168,고삼,3980168,38400
174421,35.610534,128.024177,TSB398000169,봉계,3980169,38400


## 1-3. 정류소별경유노선 목록조회

In [8]:
stop_id = city_code_df[['citycode']] # api 조회 시 필수적으로 필요한 정류소ID

def get_api_conts_3(key,city_code):
    
    url = 'https://apis.data.go.kr/1613000/BusSttnInfoInqireService/getSttnThrghRouteList?serviceKey=%s&pageNo=1&numOfRows=1000000&_type=json&cityCode=%s'%(key,city_code)
    
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    data = json_ob['response']['body']['items']['item']
    return data

In [9]:
route = []
for i in tqdm(range(len(stop_id))):
    try:
        js = get_api_conts_3(key,stop_id['citycode'][i])
        for item in js:
            route.append(item)
    except:
        pass


100%|████████████████████████████████████████████████████████████████████████████████| 132/132 [01:49<00:00,  1.20it/s]


In [10]:
route_df = pd.json_normalize(route)
route_df

,endnodenm,routeid,routeno,routetp,startnodenm
0,세종고속시외버스터미널,SJB293000024,430,간선버스,가톨릭꽃동네대학교
1,조치원역,SJB293000029,550,간선버스,산성동종점
2,산성시장,SJB293000030,551,지선버스,조치원역
3,조치원역,SJB293000033,601,간선버스,"새나루마을9,10단지"
4,세종고속시외버스터미널,SJB293000039,655,광역버스,충남대학교
...,...,...,...,...,...
19750,합천터미널,TSB398000169,660-1,농어촌(일반)버스,합천터미널
19751,합천터미널,TSB398000170,670-2,농어촌(일반)버스,합천터미널
19752,합천터미널,TSB398000171,810-3,농어촌(일반)버스,부림면정류장
19753,합천터미널,TSB398000172,110-13,농어촌(일반)버스,부림면정류장


## 1-4 컬럼명 정리 후 다운로드

In [11]:
city_code_df.columns = ['도시코드','도시명']
city_code_df.to_csv('[버스정류소정보]도시코드_목록_조회.csv')

stop_num_df.columns = ['정류소 Y좌표','정류소 X좌표','정류소 ID','정류소명','도시코드']
stop_num_df.to_csv('[버스정류소정보]정류소번호_목록.csv')

route_df.columns = ['종점','노선ID','노선번호','노선유형','기점']
route_df.to_csv('[버스정류소정보]정류소별경유노선_목록조회.csv')

ValueError: Length mismatch: Expected axis has 6 elements, new values have 5 elements